# Clean old WKCM data
Removes all HTML tags (not generalized, only the ones in the sheet at the time)   
Replaces highlighting tags with custom markup tags  
Replaces users c and ript:void(0) caused by bugs in the old version with Anonymous  
Replaces old delimiter (pipe: |) by obscure Group separator control character  

In [324]:
import pandas as pd
import requests
import re
import math
import json
import datetime
import re

In [326]:
# Download data as csv by hand and read
df = pd.read_csv("~/Documents/JavaScript/WKCM2/WKCMDB_legacy.csv", sep=',')

In [327]:
# Delete emty and unnecesarry columns
for i in range(9, 26):
    col = "Unnamed: " + str(i)
    try:
        df.drop(columns=col, inplace=True)
    except KeyError as e:
        pass
df.drop(columns="Info", inplace=True)

In [328]:
# cleanup functions
def update_users(x):
    if type(x) is float:
        return ""
    users = x.split("|")
    #print(users)
    for idx in range(len(users)):
        if users[idx] == "c" or users[idx] == "ript:void(0)":
            users[idx] = "Anonymous"
    x = users
    # x = separator.join(users)
    return x

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def update_score(x):
    if type(x) is float:
        if math.isnan(x):
            x = ""
        else:
            x = int(x)

    x = str(x)

    x = x.split("|")
    # x = separator.join(x)

    return x

def update_mnem(x):
    if type(x) is float:
        return ""
    
    x = x.replace('</span>', '[/span]')
    x = x.replace('<span class="highlight-kanji">', '[kan]')
    x = x.replace('<span class="kanji-highlight">', '[kan]')
    x = x.replace('<span class="highlight-vocabulary">', '[voc]')
    x = x.replace('<span class="vocabulary-highlight">', '[voc]')
    x = x.replace('<span class="highlight-radical">', '[rad]')
    x = x.replace('<span class="radical-highlight">', '[rad]')
    x = x.replace('<span class="highlight-reading">', '[read]')
    x = x.replace('<span class="reading-highlight">', '[read]')
    
    x = x.replace("<b>", "[b]")
    x = x.replace("</b>", "[/b]")
    x = x.replace("<i>", "[i]")
    x = x.replace("</i>", "[/i]")
    x = x.replace("<u>", "[u]")
    x = x.replace("</u>", "[/u]")
    x = x.replace("<s>", "[s]")
    x = x.replace("</s>", "[/s]")
    # x = x.replace("<img", "[img")
    x = x.replace("<br>", "\n")
    x = x.replace("<br/>", "\n")
    x = x.replace("</br>", "\n")
    x = x.replace("<br />", "\n")
    
    # only replaces the first img tag. All after that are deleted
    img_content = x[x.find(start:='<img')+len(start):x.find('>')]
    pattern = "\<img(.*?)\>"
    substring = re.search(pattern, x)
    if substring:
        substring = substring.group(1)
        x = re.sub('\<img(.*?)\>', '[img asdkljfasi]', x)
        x = x.replace("asdkljfasi", substring)
    
    x = remove_html_tags(x)
    
    x = x.split("|")
    # x = separator.join(x)
    return x


In [329]:
# actual cleanup of columns
df["Meaning_User"] = df["Meaning_User"].apply(update_users)
df["Reading_User"] = df["Reading_User"].apply(update_users)

df["Reading_Mnem"] = df["Reading_Mnem"].apply(update_mnem)
df["Meaning_Mnem"] = df["Meaning_Mnem"].apply(update_mnem)

df["Meaning_Score"] = df["Meaning_Score"].apply(update_score)
df["Reading_Score"] = df["Reading_Score"].apply(update_score)

In [330]:
# delete entries without mnem or request
for index, row in df.iterrows():
    if row['Reading_Mnem'] == "" and row['Meaning_Mnem'] == "":
        df.drop(index, inplace=True)
df.reset_index(drop=True, inplace=True)

### convert X_Mnem & X_User to combined json
New Meaning_Mnem form:
```JSON
{
    "Dakes": ["This is a mnemonic", "maybe a second one"], 
    "Anonymous": ["Anon's mnem"]
}
```
New Meaning_Mnem requested form:
```JSON
{
    "!": ["Dakes", "Anonymous", "DerTester"]
}
```
With the data formatted like this, the Meaning_User and Reading_User columns can be dropped.  
This would also allow one person to submit multiple mnemonics. 

In [331]:
# convert Meaning_Mnem & Meaning_User to combined json
types = ["Meaning", "Reading"]
for index, row in df.iterrows():
    for typ in types:
        if row[typ+'_Mnem'] == "":
            continue
            
        # Requested case
        if row[typ+'_Mnem'][0] == "!":
            if not row[typ+'_User'] or row[typ+'_User'] == [] or row[typ+'_User'] == [""]:
                row[typ+'_Mnem'] = ""
                continue
            req_dict = { "!": list(dict.fromkeys(row[typ+'_User']))}
            row[typ+'_Mnem'] = json.dumps(req_dict)
        # Mnemonic case
        else:
            mnem_dict = {}
            
            # check for case if mnem exists, but users is empty (only a few like this exist)
            if not row[typ+'_User'] and row[typ+'_Mnem'][0]:
                mnem_dict["Anonymous"] = [ row[typ+'_Mnem'][0] ]
            else:
                for idx, user in enumerate(row[typ+'_User']):
                    mnem_dict[user] = [ row[typ+'_Mnem'][idx] ]
            if mnem_dict:
                row[typ+'_Mnem'] = json.dumps(mnem_dict)
    

In [332]:
# delete entries without mnem or request /again/
for index, row in df.iterrows():
    if not row['Reading_Mnem'] and not row['Meaning_Mnem']:
        df.drop(index, inplace=True)
df.reset_index(drop=True, inplace=True)

### convert votes to json

```JSON
{
    "Dakes": [   // votes received by Dakes. Cannot contain Dakes
        { "Anonymous": 5, "DerTester": -1 },   // votes for first mnem by Dakes
        { "Anonymous": 3, "DerTester": 1 }   // votes for second mnem by Dakes
             ]
}
```

In [350]:
# Add new votes columns
df1 = pd.DataFrame(columns=['Meaning_Votes','Reading_Votes'])
df = df.join(df1, how="outer").fillna("")

types = ["Meaning", "Reading"]
for index, row in df.iterrows():
    for typ in types:
        # Requested case
        if not row[typ+'_Score']:
            continue
        # users_submitted = json.loads(row[typ+'_Mnem']).keys()
        if not row[typ+'_Mnem']:
            continue
        # print(row[typ+'_Mnem'])
        try:
            users_submitted = list( json.loads(row[typ+'_Mnem']) )
        except TypeError as err:
            print(row)
            print()
            break
        # print(row)
        if users_submitted[0] == "!":
            continue
        
        score_dict = {}
        for idx, score in enumerate(row[typ+'_Score']):
            if score == 0 or score == "0":
                continue
            try:
                current_user_submitted = users_submitted[idx]
            except IndexError as err:
                current_user_submitted = "Anonymous"
            # print(current_user_submitted)
            score_dict[current_user_submitted] = [{"Anonymous": score}]  # currently only 1 mnem / user
        if not score_dict:
            continue
        row[typ+'_Votes'] = json.dumps(score_dict)
            

### Change Last_Updated date string to not psychopathic format

In [352]:
for index, row in df.iterrows():
    if row["Last_Updated"]:
        date_str = row["Last_Updated"][:34].strip()
        # print(row["Last_Updated"])
        # print(date_str)
        # print()
        row["Last_Updated"] = datetime.datetime.strptime(date_str, '%a %b %d %Y %H:%M:%S GMT%z').strftime('%Y-%m-%d_%H-%M-%S')

### Drop unused columns

In [353]:
df.drop(columns="Meaning_Score", inplace=True)
df.drop(columns="Reading_Score", inplace=True)
df.drop(columns="Meaning_User", inplace=True)
df.drop(columns="Reading_User", inplace=True)

### split item into type and item
v被る -> v | 被る

In [354]:
df1 = pd.DataFrame(columns=['Type'])
df = df.join(df1, how="outer").fillna("")

for index, row in df.iterrows():
    old_item = row["Item"]
    row["Item"] = old_item[1:]
    row["Type"] = old_item[:1]

In [355]:
df

,Item,Meaning_Mnem,Reading_Mnem,Last_Updated,Meaning_Votes,Reading_Votes,Type
0,夕,,"{""!"": [""Anonymous""]}",2020-05-05_12-59-03,,,k
1,被る,"{""!"": [""xpetersx""]}","{""fioraaeterna"": [""Why do you [voc]put on a ha...",2020-09-17_14-28-25,,"{""fioraaeterna"": [{""Anonymous"": ""2""}]}",v
2,小皿,"{""!"": [""ExtremeWQ""]}","{""Anonymous"": [""If you're an adult, you get th...",2020-12-15_18-25-53,,"{""Anonymous"": [{""Anonymous"": ""1""}]}",v
3,岡,,"{""!"": [""Anonymous""]}",2020-12-29_10-38-32,,,k
4,努,"{""!"": [""ExtremeWQ""]}","{""!"": [""ExtremeWQ""]}",2017-05-21_19-47-30,,,k
...,...,...,...,...,...,...,...
984,捉,,"{""Anonymous"": [""Eenie meenie miney moe, CATCH ...",2020-04-11_01-40-40,,,k
985,斤,,"{""Anonymous"": [""The [b]kin[/b] g guard are al...",2019-09-11_11-25-32,,,k
986,袖,,"{""!"": [""Anonymous""]}",2020-02-06_13-15-44,,,k
987,羨む,,"{""!"": [""Anonymous""]}",2020-10-30_12-38-03,,,v


## Upload to google sheet
requires a google sheets api access and service account credentials in *.json

In [360]:

import gspread
from gspread_dataframe import set_with_dataframe

In [361]:
# ACCES GOOGLE SHEET
gc = gspread.service_account(filename='/home/dakes/Documents/JavaScript/WKCM2/wkcm2-332109-975b83d32a97.json')
sh = gc.open_by_key('13oZkp8eS059nxsYc6fOJNC3PjXVnFvUC8ntRt8fdoCs')
worksheet = sh.get_worksheet(3) #-> 0 - first sheet, 1 - second sheet etc. 

In [362]:
# APPEND DATA TO SHEET
#your_dataframe = pd.DataFrame()
set_with_dataframe(worksheet, df) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET

# after upload
- create columns Reading_Score, Meaning_Score
- Sort columns: Type, Item, Meaning_Mnem, Meaning_Votes, Meaning_Score, Reading_Mnem, Reading_Votes, Reading_Score, Last_Updated
- Add custom score calculation formula from apps script to Reading_Score, Meaning_Score